In [8]:
import logging
logger = logging.getLogger()
logger.setLevel(logging.NOTSET)

In [9]:
from dataset import UserRatingsDataset
from torch.utils.data import DataLoader

cutoff = lambda x: 1.0 if x >= 3.5 else 0.0 # function for generating implicit ratings

dataset = UserRatingsDataset("../data/ml-10m/ratings.csv",threshold=50, rating_function=cutoff)
train, valid, test = dataset.tvt_datasets()

train = DataLoader(train, batch_size=100, num_workers=12)
valid = DataLoader(valid, batch_size=100, num_workers=12)
test = DataLoader(test, batch_size=100, num_workers=12)

DEBUG:data.dataset:Loaded pandas dataframe into memory
DEBUG:data.dataset:Applying filter strategies to dataset
DEBUG:data.dataset:Finished applying filter strategies to dataset
DEBUG:data.dataset:Finished cleaning
INFO:data.dataset:Created surprise dataset
INFO:data.dataset:Finished building surprise trainset


In [10]:
from model import Model
model = Model(dataset.n_items, 200)

In [11]:
import pytorch_lightning as pl
from pytorch_lightning.callbacks import EarlyStopping

trainer = pl.Trainer(
    max_epochs=200,
    log_every_n_steps=20,
    callbacks=[EarlyStopping(monitor="ndcg@100", mode="max", patience=10)],
)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [12]:
trainer.fit(model, train, valid)

INFO:pytorch_lightning.callbacks.model_summary:
  | Name    | Type    | Params
------------------------------------
0 | drop    | Dropout | 0     
1 | encoder | Encoder | 6.6 M 
2 | decoder | Decoder | 6.5 M 
------------------------------------
13.2 M    Trainable params
0         Non-trainable params
13.2 M    Total params
52.729    Total estimated model params size (MB)
DEBUG:fsspec.local:open file: /home/nikola/projects/deepRec/vae/lightning_logs/version_9/hparams.yaml


Epoch 0: 100%|██████████| 349/349 [00:34<00:00, 10.18it/s, v_num=9]        

DEBUG:fsspec.local:open file: /home/nikola/projects/deepRec/vae/lightning_logs/version_9/hparams.yaml
DEBUG:fsspec.local:open file: /home/nikola/projects/deepRec/vae/lightning_logs/version_9/metrics.csv


Epoch 0: 100%|██████████| 349/349 [00:53<00:00,  6.54it/s, v_num=9, ndcg@100=0.547, hr@1=0.794, hr@10=1.000]

DEBUG:fsspec.local:open file: /home/nikola/projects/deepRec/vae/lightning_logs/version_9/checkpoints/epoch=0-step=349.ckpt


Epoch 1: 100%|██████████| 349/349 [00:37<00:00,  9.19it/s, v_num=9, ndcg@100=0.547, hr@1=0.794, hr@10=1.000]

DEBUG:fsspec.local:open file: /home/nikola/projects/deepRec/vae/lightning_logs/version_9/hparams.yaml
DEBUG:fsspec.local:open file: /home/nikola/projects/deepRec/vae/lightning_logs/version_9/metrics.csv


Epoch 1: 100%|██████████| 349/349 [00:58<00:00,  5.95it/s, v_num=9, ndcg@100=0.522, hr@1=0.725, hr@10=1.000]

DEBUG:fsspec.local:open file: /home/nikola/projects/deepRec/vae/lightning_logs/version_9/checkpoints/epoch=1-step=698.ckpt


Epoch 2:  42%|████▏     | 148/349 [00:16<00:22,  8.81it/s, v_num=9, ndcg@100=0.522, hr@1=0.725, hr@10=1.000]

In [6]:
trainer.test(model, test)

DEBUG:fsspec.local:open file: /home/nikola/projects/deepRec/vae/lightning_logs/version_8/hparams.yaml
DEBUG:fsspec.local:open file: /home/nikola/projects/deepRec/vae/lightning_logs/version_8/metrics.csv


Testing DataLoader 0: 100%|██████████| 5/5 [00:00<00:00, 10.35it/s]

DEBUG:fsspec.local:open file: /home/nikola/projects/deepRec/vae/lightning_logs/version_8/hparams.yaml
DEBUG:fsspec.local:open file: /home/nikola/projects/deepRec/vae/lightning_logs/version_8/metrics.csv


Testing DataLoader 0: 100%|██████████| 5/5 [00:00<00:00, 10.23it/s]

DEBUG:fsspec.local:open file: /home/nikola/projects/deepRec/vae/lightning_logs/version_8/hparams.yaml
DEBUG:fsspec.local:open file: /home/nikola/projects/deepRec/vae/lightning_logs/version_8/metrics.csv



────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
          hr@1              0.4651162922382355
          hr@10                     1.0
        ndcg@100            0.43313759565353394
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'ndcg@100': 0.43313759565353394, 'hr@1': 0.4651162922382355, 'hr@10': 1.0}]